# SQL Module Test - Solutions

This test covers all topics from the SQL module:
- SQLite basics (connecting, creating tables)
- CRUD operations (INSERT, SELECT, UPDATE, DELETE)
- Advanced queries (WHERE, ORDER BY, GROUP BY, JOINs)
- Pandas SQL integration (read_sql, to_sql)

**Instructions:**
1. Run the setup cell below to create the test database
2. Answer each question in the provided code cell
3. Each question specifies what output or result is expected
4. Do not modify the setup cell

## Setup - Run This Cell First

This creates the test database with sample data.

In [ ]:
import sqlite3
import pandas as pd
from datetime import date

# Create an in-memory database for testing
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.executescript('''
    CREATE TABLE departments (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        budget REAL NOT NULL
    );
    
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department_id INTEGER,
        salary REAL NOT NULL,
        hire_date TEXT NOT NULL,
        FOREIGN KEY (department_id) REFERENCES departments (id)
    );
    
    CREATE TABLE projects (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        department_id INTEGER,
        start_date TEXT NOT NULL,
        end_date TEXT,
        FOREIGN KEY (department_id) REFERENCES departments (id)
    );
''')

# Insert sample data - Departments
departments_data = [
    (1, 'Engineering', 500000),
    (2, 'Marketing', 200000),
    (3, 'Sales', 300000),
    (4, 'Human Resources', 150000),
    (5, 'Research', 400000)
]
cursor.executemany('INSERT INTO departments VALUES (?, ?, ?)', departments_data)

# Insert sample data - Employees
employees_data = [
    (1, 'Alice Johnson', 1, 95000, '2020-03-15'),
    (2, 'Bob Smith', 1, 85000, '2021-06-01'),
    (3, 'Carol White', 2, 72000, '2019-11-20'),
    (4, 'David Brown', 3, 68000, '2022-01-10'),
    (5, 'Eva Martinez', 1, 110000, '2018-05-22'),
    (6, 'Frank Wilson', 2, 65000, '2023-02-14'),
    (7, 'Grace Lee', 3, 78000, '2020-09-30'),
    (8, 'Henry Taylor', 4, 55000, '2021-04-05'),
    (9, 'Ivy Chen', 5, 92000, '2019-08-12'),
    (10, 'Jack Davis', 5, 88000, '2022-07-18'),
    (11, 'Karen Miller', 1, 78000, '2023-01-25'),
    (12, 'Leo Garcia', 3, 71000, '2020-12-01')
]
cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?)', employees_data)

# Insert sample data - Projects
projects_data = [
    (1, 'Website Redesign', 1, '2023-01-01', '2023-06-30'),
    (2, 'Mobile App', 1, '2023-03-15', '2023-12-31'),
    (3, 'Marketing Campaign', 2, '2023-02-01', '2023-04-30'),
    (4, 'Sales Training', 3, '2023-04-01', '2023-05-15'),
    (5, 'AI Research', 5, '2023-01-01', None),
    (6, 'Product Launch', 2, '2023-06-01', '2023-08-31'),
    (7, 'Data Pipeline', 1, '2023-07-01', None)
]
cursor.executemany('INSERT INTO projects VALUES (?, ?, ?, ?, ?)', projects_data)

conn.commit()
print("Database setup complete!")
print(f"Tables created: departments, employees, projects")
print(f"Departments: {len(departments_data)} rows")
print(f"Employees: {len(employees_data)} rows")
print(f"Projects: {len(projects_data)} rows")

---
## Part 1: SQLite Basics (Questions 1-3)

### Question 1: Exploring Table Structure

Write a query to retrieve the schema information for the `employees` table using SQLite's `PRAGMA table_info()` command.

Print the column names and their data types.

In [ ]:
# Solution
cursor.execute("PRAGMA table_info(employees)")
columns = cursor.fetchall()

print("Employees Table Schema:")
print("-" * 40)
print(f"{'Column Name':<20} {'Data Type':<15}")
print("-" * 40)

for col in columns:
    # col format: (cid, name, type, notnull, dflt_value, pk)
    col_name = col[1]
    col_type = col[2]
    print(f"{col_name:<20} {col_type:<15}")

### Question 2: Creating a New Table

Create a new table called `skills` with the following structure:
- `id`: INTEGER, primary key
- `employee_id`: INTEGER, foreign key referencing employees(id)
- `skill_name`: TEXT, not null
- `proficiency_level`: INTEGER (1-5)

After creating the table, verify it exists by listing all tables in the database.

In [ ]:
# Solution
cursor.execute('''
    CREATE TABLE skills (
        id INTEGER PRIMARY KEY,
        employee_id INTEGER,
        skill_name TEXT NOT NULL,
        proficiency_level INTEGER CHECK(proficiency_level BETWEEN 1 AND 5),
        FOREIGN KEY (employee_id) REFERENCES employees (id)
    )
''')
conn.commit()

# Verify table exists
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

print("Tables in database:")
for table in tables:
    print(f"  - {table[0]}")

### Question 3: Database Connection Context Manager

Write a function called `get_employee_count()` that:
1. Creates a new connection to an in-memory SQLite database
2. Creates an employees table with columns: id (INTEGER PRIMARY KEY), name (TEXT)
3. Inserts 3 sample employees
4. Returns the total count of employees
5. Uses the connection as a context manager to ensure proper cleanup

Call the function and print the result.

In [ ]:
# Solution
def get_employee_count() -> int:
    """Create a database, add employees, and return the count."""
    with sqlite3.connect(':memory:') as conn:
        cursor = conn.cursor()
        
        # Create table
        cursor.execute('''
            CREATE TABLE employees (
                id INTEGER PRIMARY KEY,
                name TEXT
            )
        ''')
        
        # Insert sample employees
        employees = [
            (1, 'John Doe'),
            (2, 'Jane Smith'),
            (3, 'Bob Johnson')
        ]
        cursor.executemany('INSERT INTO employees VALUES (?, ?)', employees)
        conn.commit()
        
        # Get count
        cursor.execute('SELECT COUNT(*) FROM employees')
        count = cursor.fetchone()[0]
        
        return count

# Call the function
result = get_employee_count()
print(f"Total employee count: {result}")

---
## Part 2: CRUD Operations (Questions 4-6)

### Question 4: INSERT - Adding New Records

Insert a new department called 'Finance' with a budget of 250000.

Then insert two new employees into this Finance department:
- 'Maria Santos', salary 82000, hire_date '2023-06-15'
- 'Nathan Park', salary 76000, hire_date '2023-08-01'

Use parameterized queries to prevent SQL injection. Print the new records to verify.

In [ ]:
# Solution

# Insert Finance department
cursor.execute(
    'INSERT INTO departments (name, budget) VALUES (?, ?)',
    ('Finance', 250000)
)
finance_id = cursor.lastrowid
print(f"Inserted Finance department with ID: {finance_id}")

# Insert new employees
new_employees = [
    ('Maria Santos', finance_id, 82000, '2023-06-15'),
    ('Nathan Park', finance_id, 76000, '2023-08-01')
]

cursor.executemany(
    'INSERT INTO employees (name, department_id, salary, hire_date) VALUES (?, ?, ?, ?)',
    new_employees
)
conn.commit()

# Verify the new records
print("\nNew Department:")
cursor.execute('SELECT * FROM departments WHERE name = ?', ('Finance',))
print(cursor.fetchone())

print("\nNew Employees:")
cursor.execute('SELECT * FROM employees WHERE department_id = ?', (finance_id,))
for emp in cursor.fetchall():
    print(emp)

### Question 5: UPDATE - Modifying Records

The Engineering department has received a budget increase. Update the budget to 600000.

Also, give all employees in the Engineering department a 10% raise.

Print the updated department and employee records to verify the changes.

In [ ]:
# Solution

# Update Engineering department budget
cursor.execute(
    'UPDATE departments SET budget = ? WHERE name = ?',
    (600000, 'Engineering')
)
print(f"Updated {cursor.rowcount} department(s)")

# Get Engineering department ID
cursor.execute('SELECT id FROM departments WHERE name = ?', ('Engineering',))
eng_id = cursor.fetchone()[0]

# Give 10% raise to Engineering employees
cursor.execute(
    'UPDATE employees SET salary = salary * 1.10 WHERE department_id = ?',
    (eng_id,)
)
print(f"Updated {cursor.rowcount} employee(s) with 10% raise")

conn.commit()

# Verify changes
print("\nUpdated Engineering Department:")
cursor.execute('SELECT * FROM departments WHERE name = ?', ('Engineering',))
print(cursor.fetchone())

print("\nUpdated Engineering Employees (with new salaries):")
cursor.execute('''
    SELECT name, salary 
    FROM employees 
    WHERE department_id = ?
    ORDER BY salary DESC
''', (eng_id,))
for emp in cursor.fetchall():
    print(f"  {emp[0]}: ${emp[1]:,.2f}")

### Question 6: DELETE - Removing Records

Delete all projects that have already ended (end_date is not NULL and end_date < '2023-07-01').

Print how many projects were deleted and list the remaining projects.

In [ ]:
# Solution

# First, let's see what will be deleted
cursor.execute('''
    SELECT name, end_date 
    FROM projects 
    WHERE end_date IS NOT NULL AND end_date < '2023-07-01'
''')
to_delete = cursor.fetchall()
print("Projects to be deleted:")
for proj in to_delete:
    print(f"  - {proj[0]} (ended: {proj[1]})")

# Delete the projects
cursor.execute('''
    DELETE FROM projects 
    WHERE end_date IS NOT NULL AND end_date < '2023-07-01'
''')
deleted_count = cursor.rowcount
conn.commit()

print(f"\nDeleted {deleted_count} project(s)")

# List remaining projects
print("\nRemaining Projects:")
cursor.execute('SELECT id, name, start_date, end_date FROM projects')
for proj in cursor.fetchall():
    end_status = proj[3] if proj[3] else 'Ongoing'
    print(f"  {proj[0]}. {proj[1]} (started: {proj[2]}, end: {end_status})")

---
## Part 3: Advanced Queries (Questions 7-10)

### Question 7: Filtering with WHERE and ORDER BY

Write a query to find all employees who:
- Were hired after January 1, 2020
- Have a salary greater than 70000

Order the results by salary in descending order.

Display the employee name, salary, and hire_date.

In [ ]:
# Solution
cursor.execute('''
    SELECT name, salary, hire_date
    FROM employees
    WHERE hire_date > '2020-01-01' AND salary > 70000
    ORDER BY salary DESC
''')

results = cursor.fetchall()

print("Employees hired after 2020-01-01 with salary > $70,000:")
print("-" * 55)
print(f"{'Name':<20} {'Salary':>12} {'Hire Date':>15}")
print("-" * 55)

for row in results:
    print(f"{row[0]:<20} ${row[1]:>10,.2f} {row[2]:>15}")

### Question 8: GROUP BY with Aggregate Functions

Write a query that shows for each department:
- Department name
- Number of employees
- Average salary (rounded to 2 decimal places)
- Highest salary
- Lowest salary

Only include departments that have more than 1 employee.
Order by average salary descending.

In [ ]:
# Solution
cursor.execute('''
    SELECT 
        d.name AS department,
        COUNT(e.id) AS employee_count,
        ROUND(AVG(e.salary), 2) AS avg_salary,
        MAX(e.salary) AS max_salary,
        MIN(e.salary) AS min_salary
    FROM departments d
    JOIN employees e ON d.id = e.department_id
    GROUP BY d.id, d.name
    HAVING COUNT(e.id) > 1
    ORDER BY avg_salary DESC
''')

results = cursor.fetchall()

print("Department Salary Statistics (departments with >1 employee):")
print("-" * 75)
print(f"{'Department':<18} {'Employees':>10} {'Avg Salary':>14} {'Max Salary':>14} {'Min Salary':>14}")
print("-" * 75)

for row in results:
    print(f"{row[0]:<18} {row[1]:>10} ${row[2]:>12,.2f} ${row[3]:>12,.2f} ${row[4]:>12,.2f}")

### Question 9: INNER JOIN

Write a query using INNER JOIN to display:
- Employee name
- Department name
- Project name(s) they are working on (based on their department)

Only show employees whose departments have active projects (projects with no end_date).

In [ ]:
# Solution
cursor.execute('''
    SELECT 
        e.name AS employee_name,
        d.name AS department_name,
        p.name AS project_name
    FROM employees e
    INNER JOIN departments d ON e.department_id = d.id
    INNER JOIN projects p ON d.id = p.department_id
    WHERE p.end_date IS NULL
    ORDER BY d.name, e.name
''')

results = cursor.fetchall()

print("Employees working on active projects (no end date):")
print("-" * 70)
print(f"{'Employee':<20} {'Department':<18} {'Project':<25}")
print("-" * 70)

for row in results:
    print(f"{row[0]:<20} {row[1]:<18} {row[2]:<25}")

### Question 10: LEFT JOIN and Subqueries

Write a query to find all departments and their total employee salary cost.

Use a LEFT JOIN to ensure departments with no employees are also included (showing 0 for salary cost).

Also include a column showing if the total salary cost exceeds 50% of the department's budget (show 'Over Budget Risk' or 'Within Budget').

In [ ]:
# Solution
cursor.execute('''
    SELECT 
        d.name AS department,
        d.budget,
        COALESCE(SUM(e.salary), 0) AS total_salary_cost,
        CASE 
            WHEN COALESCE(SUM(e.salary), 0) > (d.budget * 0.5) 
            THEN 'Over Budget Risk'
            ELSE 'Within Budget'
        END AS budget_status
    FROM departments d
    LEFT JOIN employees e ON d.id = e.department_id
    GROUP BY d.id, d.name, d.budget
    ORDER BY total_salary_cost DESC
''')

results = cursor.fetchall()

print("Department Salary Cost Analysis:")
print("-" * 80)
print(f"{'Department':<18} {'Budget':>14} {'Total Salary':>14} {'Status':<20}")
print("-" * 80)

for row in results:
    print(f"{row[0]:<18} ${row[1]:>12,.2f} ${row[2]:>12,.2f} {row[3]:<20}")

---
## Part 4: Pandas SQL Integration (Questions 11-14)

### Question 11: Reading SQL Data into Pandas

Use `pd.read_sql()` to load the employees table into a DataFrame.

Then use pandas methods to:
1. Display basic statistics for the salary column
2. Show the distribution of employees by department_id

In [ ]:
# Solution

# Load employees into DataFrame
df_employees = pd.read_sql('SELECT * FROM employees', conn)

print("Employees DataFrame:")
print(df_employees.head())
print()

# 1. Basic statistics for salary
print("Salary Statistics:")
print(df_employees['salary'].describe())
print()

# 2. Distribution by department_id
print("Employee Distribution by Department:")
print(df_employees['department_id'].value_counts().sort_index())

### Question 12: Complex Query with Pandas

Use `pd.read_sql()` with a JOIN query to create a DataFrame containing:
- Employee name
- Department name
- Salary
- Department budget

Add a new calculated column called 'salary_to_budget_ratio' showing what percentage of the department budget each employee's salary represents.

Display the top 5 employees by this ratio.

In [ ]:
# Solution

# Query with JOIN
query = '''
    SELECT 
        e.name AS employee_name,
        d.name AS department_name,
        e.salary,
        d.budget
    FROM employees e
    JOIN departments d ON e.department_id = d.id
'''

df = pd.read_sql(query, conn)

# Calculate salary to budget ratio (as percentage)
df['salary_to_budget_ratio'] = (df['salary'] / df['budget'] * 100).round(2)

# Display top 5 by ratio
print("Top 5 Employees by Salary-to-Budget Ratio:")
print("-" * 85)
top_5 = df.nlargest(5, 'salary_to_budget_ratio')
print(top_5.to_string(index=False))

### Question 13: Writing DataFrame to SQL

Create a pandas DataFrame with the following performance review data:

| employee_id | review_date | rating | comments |
|-------------|-------------|--------|----------|
| 1 | 2023-06-15 | 5 | Excellent work on the website redesign |
| 2 | 2023-06-15 | 4 | Strong technical skills |
| 3 | 2023-06-15 | 4 | Great marketing campaigns |
| 5 | 2023-06-15 | 5 | Outstanding leadership |

Use `to_sql()` to write this DataFrame to a new table called 'performance_reviews'.

Then query the table to verify the data was written correctly.

In [ ]:
# Solution

# Create DataFrame with performance review data
reviews_data = {
    'employee_id': [1, 2, 3, 5],
    'review_date': ['2023-06-15', '2023-06-15', '2023-06-15', '2023-06-15'],
    'rating': [5, 4, 4, 5],
    'comments': [
        'Excellent work on the website redesign',
        'Strong technical skills',
        'Great marketing campaigns',
        'Outstanding leadership'
    ]
}

df_reviews = pd.DataFrame(reviews_data)
print("Performance Reviews DataFrame:")
print(df_reviews)
print()

# Write to SQL table
df_reviews.to_sql('performance_reviews', conn, if_exists='replace', index=False)
print("Data written to 'performance_reviews' table.")
print()

# Verify by querying the table
print("Verification - Reading back from database:")
df_verify = pd.read_sql('SELECT * FROM performance_reviews', conn)
print(df_verify)

### Question 14: Pandas SQL Analysis Challenge

Using pandas and SQL together, answer this business question:

**"Which department has the best return on investment in terms of projects per dollar of salary spent?"**

Calculate:
1. Total salary cost per department
2. Number of projects per department
3. Projects per $100,000 of salary cost

Display the results sorted by efficiency (projects per salary cost) descending.

Hint: You may need multiple queries or joins, and you can use pandas for the final calculations.

In [ ]:
# Solution

# Query to get department salary costs
salary_query = '''
    SELECT 
        d.id AS department_id,
        d.name AS department_name,
        COALESCE(SUM(e.salary), 0) AS total_salary
    FROM departments d
    LEFT JOIN employees e ON d.id = e.department_id
    GROUP BY d.id, d.name
'''

# Query to get project counts per department
project_query = '''
    SELECT 
        d.id AS department_id,
        d.name AS department_name,
        COUNT(p.id) AS project_count
    FROM departments d
    LEFT JOIN projects p ON d.id = p.department_id
    GROUP BY d.id, d.name
'''

# Load into DataFrames
df_salary = pd.read_sql(salary_query, conn)
df_projects = pd.read_sql(project_query, conn)

# Merge the DataFrames
df_analysis = pd.merge(
    df_salary,
    df_projects[['department_id', 'project_count']],
    on='department_id'
)

# Calculate projects per $100,000 of salary
# Handle division by zero for departments with no employees
df_analysis['projects_per_100k'] = df_analysis.apply(
    lambda row: (row['project_count'] / row['total_salary'] * 100000) 
    if row['total_salary'] > 0 else 0,
    axis=1
).round(4)

# Sort by efficiency
df_analysis = df_analysis.sort_values('projects_per_100k', ascending=False)

# Display results
print("Department ROI Analysis: Projects per $100,000 of Salary Cost")
print("=" * 75)
print()
print(f"{'Department':<18} {'Total Salary':>14} {'Projects':>10} {'Proj/$100K':>12}")
print("-" * 75)

for _, row in df_analysis.iterrows():
    print(f"{row['department_name']:<18} ${row['total_salary']:>12,.2f} {row['project_count']:>10} {row['projects_per_100k']:>12.4f}")

print()
best_dept = df_analysis.iloc[0]
print(f"Best ROI: {best_dept['department_name']} with {best_dept['projects_per_100k']:.4f} projects per $100K salary")

---
## Cleanup

In [ ]:
# Close the database connection when done
conn.close()
print("Database connection closed.")

---
## End of Test

Make sure you have answered all 14 questions before submitting.